## 태스크 2: 모델 튜닝

데이터 과학자의 전형적인 기계 학습(ML) 프로세스는 5가지 주요 단계인 데이터 준비, 모델 훈련, 모델 튜닝, 엔드포인트로 모델 배포, 프로덕션의 모델 평가하여 필요에 따라 재훈련으로 구성됩니다. 이전 실습에서는 데이터 준비 및 모델 훈련 단계를 완료했습니다. 이 실습에서는 모델 튜닝을 완료합니다. 서비스 내에서 강력한 Jupyter Notebook 인스턴스, 기본 제공 알고리즘, 모델 훈련 및 모델 배포에 액세스하여 SageMaker Studio에서 이러한 각 단계를 완료할 수 있습니다. 

프로세스의 훈련 및 튜닝 부분 동안 일반적으로 데이터를 사용하고 모델에 제공하여 예측된 결과를 기준으로 모델의 예측을 평가합니다. 입력 데이터, 테스트 데이터 부분은 모델의 훈련 및 튜닝에 사용하는 훈련 및 검증 데이터와는 별도로 유지합니다. 다음 실습에서는 테스트 데이터를 사용하여 확인된 적이 없는 데이터에 대한 모델의 행동을 검토합니다. 이 실습의 경우 훈련 및 검증 데이터를 사용하여 모델을 튜닝합니다. 하이퍼파라미터 구성을 조정하여 모델을 튜닝합니다. 이러한 조정의 목표는 출력 지표를 점점 더 개선하는 것입니다.

Amazon SageMaker AI는 효율적인 방법으로 모델의 최적 버전을 찾기 위한 자동 모델 튜닝(하이퍼파라미터 튜닝)을 제공합니다. SageMaker AI 하이퍼파라미터 튜닝은 지정된 알고리즘 및 하이퍼파라미터 범위를 사용하여 데이터세트에 대해 많은 훈련 작업을 실행합니다. 그런 다음, 선택한 지표에 대해 결정된 최적화된 성능 모델을 제공하는 하이퍼파라미터 값을 선택합니다. 튜닝할 ML 모델, 객관적인 지표, 사용할 하이퍼파라미터를 지정하고 SageMaker AI 하이퍼파라미터 튜닝을 수행하면 비용 효과적인 방법으로 최적의 모델 버전이 확인됩니다.

### 태스크 2.1: 환경 설정

모델 튜닝을 시작하기 전에 필요한 종속 항목을 모두 설치하십시오.

In [1]:
#Install matplotlib and restart kernel
%pip install matplotlib
%pip uninstall bokeh -y
%pip install bokeh==2.4.2
%reset -f

# Install dependencies 
import boto3
import numpy as np
import pandas as pd
import sagemaker
import bokeh
import bokeh.io

from sagemaker.inputs import TrainingInput
from pprint import pprint
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost
from time import strftime
from bokeh.models import HoverTool
from bokeh.plotting import figure, show

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 137.1 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 위의 셀과 후속 셀을 모두 실행하고 나면 경고를 무시해도 됩니다. 

다음으로는 데이터세트를 가져옵니다.

In [2]:
# Import the dataset 
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)
prefix = 'scripts/data'
output_path = 's3://{}/{}/output'.format(bucket, prefix)

train_path = f"s3://{bucket}/{prefix}/train/adult_data_processed_train.csv"
validation_path = f"s3://{bucket}/{prefix}/validation/adult_data_processed_validation.csv"

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

print(f'Training path: {train_path}')
print(f'Validation path: {validation_path}')

create_date = strftime("%m%d%H%M")
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='1.2-1')
run_name = 'lab-3-run-{}'.format(create_date)
run_tags = [{'Key': 'lab-3', 'Value': 'lab-3-run'}]
job_name = 'lab-3-job-{}'.format(create_date)

Bucket:  labdatabucket-us-west-2-490266400
Training path: s3://labdatabucket-us-west-2-490266400/scripts/data/train/adult_data_processed_train.csv
Validation path: s3://labdatabucket-us-west-2-490266400/scripts/data/validation/adult_data_processed_validation.csv


모델 훈련을 시작하는 데 필요한 라이브러리 및 데이터를 성공적으로 가져왔습니다.

### 태스크 2.2: 추정기 객체 구성

이 태스크에서는 이전 실습에서 사용한 것과 동일하게 추정기 객체를 구성합니다. 튜닝 작업의 경우 주요 차이점은 하이퍼파라미터를 구성하는 방법입니다.

In [3]:
xgb_model = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count = 1, 
    instance_type ='ml.m5.xlarge',
    output_path = output_path,
    sagemaker_session = sagemaker_session,
    EnableSageMakerMetricsTimeSeries = True,
    tags = run_tags
)

추정기 객체를 구성했습니다.

### 태스크 2.3: 하이퍼파라미터 튜너 구성

기계 학습 모델에 적합한 하이퍼파라미터 값을 선택하는 일은 어려울 수 있습니다. 올바른 해답은 알고리즘 및 데이터에 따라 다릅니다. 튜닝 가능한 많은 하이퍼파라미터가 있는 알고리즘도 있습니다. 선택한 하이퍼파라미터 값에 매우 민감한 알고리즘도 있습니다. 또한 대부분의 알고리즘은 모델 적합 및 하이퍼파라미터 값 사이에서 비선형 관계가 나타납니다. Amazon SageMaker AI 자동 모델 튜닝은 하이퍼파라미터 튜닝 프로세스를 자동화하여 도움을 줍니다.

SageMaker AI 자동 모델 튜닝을 사용하려면 튜닝하도록 선택하는 각 하이퍼파라미터의 범위(사용 가능한 값 목록)를 지정합니다. SageMaker AI 자동 모델 튜닝은 다양한 하이퍼파라미터 설정을 사용해 여러 훈련 작업을 실행합니다. 그런 다음, 지정된 목표 지표에 따라 각 작업의 결과를 평가하고 이전 결과를 기준으로 추가 시도를 위한 하이퍼파라미터 설정을 선택합니다. 각 튜닝 작업에서는 최대 훈련 작업 수를 지정하며, 해당 작업 수에 도달하면 튜닝이 완료됩니다.

자동 모델 튜닝에 관한 자세한 내용은 [SageMaker AI를 사용하여 자동 모델 튜닝 수행](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)을 참조하십시오. 

설정해야 할 하이퍼파라미터 범위는 다음과 같습니다.
- **alpha**: 가중치에 사용되는 L1 정규화 항입니다. 이 값을 늘리면 가능한 과대적합이 줄어들어 모델이 덜 복잡해집니다. 절충은 모델이 관심 클래스보다 덜 민감하고 훈련 데이터세트에 덜 적합한 것입니다.
- **eta**: 과대적합 방지를 위해 업데이트에서 사용되는 단계 크기 축소 파라미터입니다. 각 부스팅 단계를 수행한 후 새 특성의 가중치를 직접 가져올 수 있습니다. 실제로 eta 파라미터는 부스팅 프로세스에서 예상 결과와 가까운 결과를 반환하도록 특성 가중치를 축소합니다.
- **max_depth**: 트리의 최대 깊이입니다. 깊이를 증가시키면 과대적합이 발생하고 깊이를 감소시키면 과소적합이 발생할 수 있습니다.
- **min_child_weight**: 하위 항목에 필요한 인스턴스 가중치의 최소 합계입니다. 트리 파티션 단계를 수행하여 생성된 리프 노드의 인스턴스 가중치 합계가 min_child_weight보다 작으면 빌드 프로세스에서는 추가 분할을 수행하지 않습니다.
- **num_round**: 부스팅에 사용되는 라운드(트리)의 수입니다. 트리 크기를 늘리면 모델 정확도는 높아질 수 있지만 과대적합 위험성도 높아집니다.

XGBoost 하이퍼파라미터에 관한 자세한 내용은 [XGBoost 하이퍼파라미터](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html)를 참조하십시오.

In [4]:
hyperparameter_ranges = {
    'alpha': ContinuousParameter(0, 2),
    'eta': ContinuousParameter(0, 1),
    'max_depth': IntegerParameter(1, 10),
    'min_child_weight': ContinuousParameter(1, 10),
    'num_round': IntegerParameter(100, 1000)
}

객관적 지표는 하이퍼파라미터 튜너가 최적화하는 데 중점을 두는 값입니다. 이 경우 검증의 곡선 아래 면적(AUC)을 최대화하려고 합니다. AUC는 부정 예제와 비교할 때 긍정 예제에 대해 더 높은 점수를 예측하는 모델의 능력을 측정합니다. AUC는 점수 컷오프와는 별개이므로 임곗값을 선택하지 않고 AUC 지표에서 모델의 예측 정확도를 파악할 수 있습니다. AUC 지표는 0~1 범위의 10진수 값을 반환합니다. AUC가 0.50인 모델은 랜덤 가능성을 나타내므로 동전 뒤집기보다 나을 바가 없지만, '완벽' 모델은 점수가 1.0이 됩니다. AUC가 높을수록 모델이 사기와 합법 간을 더 잘 구분할 수 있습니다. 0에 가까운 값은 찾기 어려우며, 일반적으로는 데이터에 문제가 있는 것을 나타냅니다. 

In [5]:
objective_metric_name = 'validation:auc'
objective_type='Maximize'

추정기 객체를 사용하여 하이퍼파라미터 튜닝 작업의 결과로 생성된 훈련 작업에 대한 구성 정보를 획득합니다. 필요한 HyperparameterTuner 파라미터는 다음과 같습니다.
- **estimator**: 필수 구성으로 초기화된 추정기 객체. 이 인스턴스와 연관된 훈련 작업은 없어도 됩니다.
- **objective_metric_name**: 훈련 작업을 평가하기 위한 지표의 이름
- **hyperparameter_ranges**: 파라미터 범위의 사전. 이러한 파라미터 범위는 Continuous, Integer 또는 Categorical의 3가지 유형 중 하나일 수 있습니다. 사전의 키는 하이퍼파라미터의 이름이고, 값은 범위를 나타내는 해당 하이퍼파라미터 범위 클래스입니다.
- **objective_type**: 훈련 작업을 평가하기 위한 객관적 지표의 유형. 이 값은 'Minimize' 또는 'Maximize'(기본값: 'Maximize')일 수 있습니다.
- **max_jobs**: 하이퍼파라미터 튜닝 작업에 대해 시작할 훈련 작업의 최대 총수. ‘Grid’ 전략에 대한 기본값은 지정되지 않았으며 다른 모든 전략에 대한 기본값은 1입니다(기본값: None).
- **max_parallel_jobs**: 시작할 병렬 훈련 작업의 최대 수(기본값: 1).
- **early_stopping_type**: 작업에 대해 조기 중지가 사용하도록 설정되었는지 여부를 지정합니다. 'Auto' 또는 'Off'(기본값: 'Off')일 수 있습니다. 'Off'로 설정되면 조기 중지가 시도되지 않습니다. 'Auto'로 설정되면 일부 훈련 작업의 조기 중지가 발생할 수 있지만 보장되지는 않습니다.

다음 코드에서는 튜너에 최소 12가지 실험(max_jobs)을 실행하며 한 번에 4개의 동시 실험(max_parallel_jobs)만 실행하도록 지시합니다. 이러한 두 하이퍼파라미터는 비용 및 훈련 시간을 모두 제어합니다.

In [6]:
tuner = HyperparameterTuner(
    estimator = xgb_model,
    objective_metric_name = objective_metric_name,
    hyperparameter_ranges = hyperparameter_ranges,
    objective_type = objective_type,
    max_jobs=12,
    max_parallel_jobs=4,
    early_stopping_type='Auto',
)

하이퍼파라미터 튜너를 구성했습니다.

### 태스크 2.4: 하이퍼파라미터 튜닝 작업 실행

지금까지 추정기 객체 및 하이퍼파라미터를 구성했으므로 모델을 튜닝할 준비가 되었습니다. fit() 메서드는 튜닝 스크립트를 시작합니다. 튜닝을 실행하려면 약 5~6분이 소요됩니다. 모델 튜닝을 시작하려면 훈련 및 검증 데이터세트로 추정기의 fit() 메서드를 호출합니다. `wait=True`를 설정하는 경우 fit() 메서드는 진행 로그를 표시하고 훈련이 완료될 때까지 기다립니다.

In [7]:
tuner.fit(
    {
        "train": train_input,
        "validation": validation_input
    },
    job_name=job_name,
    wait=True
)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.........................................................!


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 위 셀이 실행되는 동안 아래 단계를 진행하여 하이퍼파라미터 튜닝 작업 진행 상황을 모니터링합니다. 

1. AWS 콘솔로 이동한 후 왼쪽 위의 검색 표시줄에서 Amazon SageMaker AI를 검색합니다. 

2. SageMaker AI 콘솔의 왼쪽 창에서 **Training**을 선택하고 **Training jobs**를 선택합니다.  

3. **Training jobs** 목록에는 작업이 최대 12개 표시되며 표시된 작업은 한 번에 4개씩 시작됩니다. 

4. 모든 작업의 상태가 **InProgress**에서 **Completed** 또는 **Stopped**로 변경되었다면 하이퍼파라미터 튜닝 작업이 완료된 것이므로 노트북으로 돌아와 다음 태스크를 진행합니다. 

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 처리는 5~6분 정도 걸릴 수 있습니다. 

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** **Stopped** 상태의 작업이 있을 수도 있습니다. 연속하는 여러 훈련 작업에서 목표 지표가 그다지 개선되지 않은 경우 등 하이퍼파라미터 튜닝 작업의 조기 중지 조건이 충족되는 상황에서 훈련 작업이 하나 이상 중지되는 것은 정상적인 현상입니다. 

5. SageMaker AI에서 하이퍼파라미터 튜닝 작업으로 이동합니다. 이름이 **lab-3-job**으로 시작하는 튜닝 작업을 선택합니다. 

6. 이 튜닝 작업에 속해 있는 모든 튜닝 작업(최종 목표 지표 값 포함)의 보기를 확인할 수 있습니다. 

7. 이 페이지의 검토를 완료한 후 이 노트북으로 돌아옵니다. 

<i aria-hidden="true" class="fas fa-clipboard-check" style="color:#18ab4b"></i> **예상 출력:** 추정기 및 하이퍼파라미터 범위 구성이 올바르고 튜닝 작업이 올바르게 시작된 경우 다음 출력이 표시됩니다.

```plain
************************
**** EXAMPLE OUTPUT ****
************************
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
......................................................................!
```

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 위 출력에서 점선 위에 느낌표(!)가 표시되면 훈련 작업이 완료된 것입니다. *"No finished training job found associated with this estimator*" 경고가 표시되는 것은 정상적인 현상입니다. 이 메시지는 SDK에서 발생하며, 추정기의 모델 데이터가 참조되고 있지만 추정기가 아직 실행되지 않았음을 경고합니다.

하이퍼파라미터 튜닝 작업을 실행했습니다.

### 태스크 2.5: 모델 평가 및 배포의 후보로 선택

튜닝 작업을 시작한 후 *describe_tuning_job* API를 호출하여 진행 상황을 확인할 수 있습니다. *describe-tuning-job*의 출력은 튜닝 작업의 현재 상태에 대한 정보가 포함된 JSON 객체입니다. *list_training_jobs_for_tuning_job*을 호출하여 튜닝 작업이 시작한 훈련 작업의 세부 목록을 확인할 수 있습니다.

In [8]:
# Print the number of completed tuning jobs
tuning_job_result = sm.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

objective = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]
is_minimize = objective["Type"] != "Maximize"
objective_name = objective["MetricName"]

7 training jobs have completed


In [9]:
# Get the best training job
if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2026, 1, 1, 5, 45, 6, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.9197800159454346},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2026, 1, 1, 5, 45, 58, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-west-2:072776568191:training-job/lab-3-job-01010540-011-bbf992af',
 'TrainingJobName': 'lab-3-job-01010540-011-bbf992af',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2026, 1, 1, 5, 45, 9, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '1.0251099066623186',
                          'eta': '0.6033331045910224',
                          'max_depth': '2',
                          'min_child_weight': '2.1628265710613173',
                          'num_round': '596'}}


모든 훈련 작업의 하이퍼파라미터 및 목표 지표를 나열하고 목표 지표가 가장 좋은 훈련 작업을 선택할 수 있습니다.

In [10]:
# Print the tuning metrics
tuner = sagemaker.HyperparameterTuningJobAnalytics(job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 12
{'lowest': 0.8394799828529358, 'highest': 0.9197800159454346}


,alpha,eta,max_depth,min_child_weight,num_round,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
1,1.025110,0.603333,2.0,2.162827,596.0,lab-3-job-01010540-011-bbf992af,Completed,0.91978,2026-01-01 05:45:09+00:00,2026-01-01 05:45:58+00:00,49.0
0,1.219962,0.602794,2.0,2.290053,483.0,lab-3-job-01010540-012-ad5d1455,Completed,0.91972,2026-01-01 05:45:11+00:00,2026-01-01 05:46:05+00:00,54.0
10,1.892602,0.654507,2.0,2.118721,645.0,lab-3-job-01010540-002-558efe04,Stopped,0.91965,2026-01-01 05:41:56+00:00,2026-01-01 05:43:36+00:00,100.0
7,0.113893,1.000000,2.0,10.000000,1000.0,lab-3-job-01010540-005-f9ab2e25,Completed,0.91959,2026-01-01 05:43:51+00:00,2026-01-01 05:44:40+00:00,49.0
4,1.927074,0.722753,2.0,5.120524,375.0,lab-3-job-01010540-008-c2296556,Completed,0.91929,2026-01-01 05:44:05+00:00,2026-01-01 05:44:54+00:00,49.0
9,0.036049,0.621603,2.0,6.393435,849.0,lab-3-job-01010540-003-db953368,Stopped,0.91885,2026-01-01 05:42:02+00:00,2026-01-01 05:43:36+00:00,94.0
6,1.184040,0.516230,3.0,7.875171,572.0,lab-3-job-01010540-006-1eebea7d,Completed,0.91566,2026-01-01 05:43:53+00:00,2026-01-01 05:44:41+00:00,48.0
2,0.039204,0.562962,1.0,4.069006,837.0,lab-3-job-01010540-010-89e0bce5,Stopped,0.91146,2026-01-01 05:44:58+00:00,2026-01-01 05:45:29+00:00,31.0
3,1.044490,0.350126,1.0,8.643353,122.0,lab-3-job-01010540-009-f9092772,Stopped,0.90056,2026-01-01 05:44:57+00:00,2026-01-01 05:45:29+00:00,32.0
5,1.323199,1.000000,4.0,9.999965,893.0,lab-3-job-01010540-007-b81fd929,Completed,0.89733,2026-01-01 05:43:54+00:00,2026-01-01 05:44:50+00:00,56.0


튜닝 작업이 진행됨에 따라 목표 지표가 시간에 따라 어떻게 변경되는지를 확인할 수도 있습니다. 베이지안 전략의 경우 일반적으로 결과가 개선되는 추세를 보이지만, 알고리즘이 알려진 양호한 영역의 활용과 파라미터 공간의 새로운 영역 탐색 간 균형을 맞춰야 하므로 이러한 개선 추세가 꾸준하지는 않습니다. 다음을 수행하면 검색 공간의 복잡성을 처리하기에 훈련 작업 수가 충분한지 여부를 알 수 있습니다.

In [11]:
# Plot the objective metric results against time
bokeh.io.output_notebook()

df = df.sort_values(by=['TrainingStartTime'], ascending=True)

# x = df['TrainingStartTime'].to_numpy()
x = df['TrainingStartTime']
y = df['FinalObjectiveValue'].to_numpy()

p = figure(
    title="Final Objective Value over Time", 
    width=900, height=400, 
    x_axis_label="TrainingStartTime",
    y_axis_label="FinalObjectiveValue",
    x_axis_type="datetime"
)

# add hover tool 
hover = HoverTool(tooltips=[
    ('FinalObjectiveValue', '@y'),
    ('TrainingStartTime', "@x{%T}")
    ], formatters={'@x': 'datetime'}) 
p.add_tools(hover) 


# p.circle(source=df, x="TrainingStartTime", y="FinalObjectiveValue")
p.line(x,y,color='green',line_width=2)
p.circle(x, y, fill_color ="red", line_color ="green", size=8) 

show(p)

Loading BokehJS ...

튜닝 작업을 마쳤으므로 이제 목표 지표와 튜닝하도록 선택한 개별 하이퍼파라미터 간의 상관 관계를 알고 싶을 수 있습니다. 이러한 인사이트를 확보하면 특정 하이퍼파라미터의 검색 범위를 조정하고 다른 튜닝 작업을 시작하는 것이 적절한지 여부를 결정하는 데 도움이 됩니다. 예를 들어 목표 지표와 수치형 하이퍼파라미터 간에 양의 추세가 나타나는 경우 다음 튜닝 작업에서 해당 하이퍼파라미터에 대해 더 큰 튜닝 범위를 설정할 수 있습니다.

다음 셀에서는 각 하이퍼파라미터에 대한 그래프를 그려 목표 지표와의 상관 관계를 보여줍니다.

In [12]:
# Plot each of the hyperparameter ranges with the objective metric results
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}

    x = df[hp_name].to_numpy()
    y = df['FinalObjectiveValue'].to_numpy()

    # determine line of best fit
    par = np.polyfit(x, y, 1, full=True)
    slope=par[0][0]
    intercept=par[0][1]
    y_predicted = [slope*i + intercept  for i in x]        


    p = figure(
        width=500,
        height=500,
        title="Objective vs %s" % hp_name,
        x_axis_label=hp_name,
        y_axis_label=objective_name,
        **categorical_args,
    )

    # add hover tool 
    hover = HoverTool(tooltips=[
        ('FinalObjectiveValue', '@y'),
        (hp_name, '@x')
    ]) 
    p.add_tools(hover) 

    p.circle(source=df, x=hp_name, y="FinalObjectiveValue")
    p.line(x,y_predicted,color='green', line_width=2)
    figures.append(p)


show(bokeh.layouts.Column(*figures))

잠시 차트를 살펴보십시오. 목표에 대해 양의 상관 관계가 있는 하이퍼파라미터는 어느 것입니까? 그래프의 점에 가장 잘 맞는 선을 사용해서 양의 상관 관계가 있는 그래프를 평가할 수 있습니다. 최적 선이 증가하는 경우 하이퍼파라미터와 목표 간에는 양의 상관 관계가 있습니다. 이러한 그래프에는 최적 선이 추가되었습니다. 

하이퍼파라미터는 이러한 튜닝 작업에서 따로 작동하지 않습니다. 각 튜닝 작업은 여러 하이퍼파라미터를 동시에 조정했습니다. 하이퍼파라미터를 분리하여 대상과의 직접적인 상관 관계를 확인할 수 있지만, 각 튜닝 작업의 결과가 여러 하이퍼파라미터 조정을 조합한 것이라는 점에 유의하십시오.

### 태스크 2.6: 모델 아티팩트 확인

SageMaker AI는 환경 설정 시에 지정한 S3 버킷에 모델 아티팩트를 저장합니다. 모델 아티팩트 중 하나의 위치를 찾으려면 다음 단계를 따르십시오.

1. 왼쪽 메뉴 모음에서 버킷 아이콘을 선택합니다.

1. 버킷 목록에서 이름에 **labdatabucket**이 포함된 Amazon S3 버킷을 엽니다.

1. **scripts/data/output/lab-3-job-.../output** 하위 폴더 중 하나로 이동합니다. 

하위 폴더에 모델 아티팩트 **model.tar.gz**가 표시됩니다. 이것은 tuner.fit() 메서드를 호출하여 SageMaker AI Estimator로 생성한 모델 중 하나입니다.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** model.tar.gz 파일은 압축된 아카이브입니다. 모델을 확인하려는 경우 S3 콘솔로 이동하여 모델을 다운로드하면 됩니다.


### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 **결론**을 계속 진행합니다.